In [5]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.commentThreads.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os

import googleapiclient.discovery

import json

def request(pageToken):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "0"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = ""

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="snippet,replies",
        pageToken = pageToken,
        videoId="5Qbd0Y-KghE"
    )
    response = request.execute()

    print(response)
    print("\n\n\n\n")
    
    # dump to json file
    
    return response["nextPageToken"]

nextPageToken = ""
for i in range(3):
    nextPageToken = request(nextPageToken)


{'kind': 'youtube#commentThreadListResponse', 'etag': 'WMjTxIgJCILPAkXZSnR_PyJLiGY', 'nextPageToken': 'QURTSl9pM3I5bmp2MnU1VUhoUDNHRGNfSGV0YWtaSHlmSFlDbG5ELVY3S3RrdkhnakthZVpxS3B1Q0JrY3luLXItU0VSV3FhSXljQU83NA==', 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20}, 'items': [{'kind': 'youtube#commentThread', 'etag': 'ba_wlX97rD94fKrXf8j1Qk43gdY', 'id': 'Ugzf0tsXguKKyUuYkod4AaABAg', 'snippet': {'videoId': '5Qbd0Y-KghE', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'jL9tuPaKMT03YZEf0wvxeB7Cafc', 'id': 'Ugzf0tsXguKKyUuYkod4AaABAg', 'snippet': {'videoId': '5Qbd0Y-KghE', 'textDisplay': '<a href="https://www.youtube.com/watch?v=5Qbd0Y-KghE&amp;t=0m14s">0:14</a><br><br>ONORE DECAIDOOO', 'textOriginal': '0:14\n\nONORE DECAIDOOO', 'authorDisplayName': 'Amido De Milho', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AKedOLRVvWZzqGuN8YMefxuxqXFMxup5W5qq_yapTPykIE0=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCYlugTdYaqlFuABw2-D8ixQ', 'authorC

{'kind': 'youtube#commentThreadListResponse', 'etag': 'IQlO5hnkWQvaiKexoOqphzhnzkU', 'nextPageToken': 'QURTSl9pMUpvakdSaEREV29OWFpkdHUtb1d0Z3JoajhhNlI3bnYxR1JyQ1l6d3BtR1JEbUoxZkUtcEl0VWdWUEdXSnY5WjZlVHhoSmNaOA==', 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20}, 'items': [{'kind': 'youtube#commentThread', 'etag': '2k6DZdBCRyHcjjk-uMbPy4BGy-s', 'id': 'UgzUUC00irhgf29RWTp4AaABAg', 'snippet': {'videoId': '5Qbd0Y-KghE', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'bBu3bK4YqOwDNjqRPXGQ6ukYF2s', 'id': 'UgzUUC00irhgf29RWTp4AaABAg', 'snippet': {'videoId': '5Qbd0Y-KghE', 'textDisplay': 'Best WAIFU ever.', 'textOriginal': 'Best WAIFU ever.', 'authorDisplayName': 'the cool zekrom', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AKedOLTkgR_M4J8rqYQ7UyheLXyYlq1w3cQba-sVLOTO=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCVY_Fpe8ALdyqf3aWTAqLHA', 'authorChannelId': {'value': 'UCVY_Fpe8ALdyqf3aWTAqLHA'}, 'canRate': True, 'viewerRating': 'none',

{'kind': 'youtube#commentThreadListResponse', 'etag': 'EujGNU2TVSznHdMipWLELCH0i80', 'nextPageToken': 'QURTSl9pMUZRdDVMZmJhM0x3WUE1TkRzWl93MU5peWYwQ1haWUlxbGN4RUJSV3p2c2l1VzUtaC1sbXQycmZVYXdOT1VxOXJZaG51b0RlQQ==', 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20}, 'items': [{'kind': 'youtube#commentThread', 'etag': 'a8qb9vTMpIJSwjsk5x5KpKZza24', 'id': 'Ugw51bV4A9ySSp5yULJ4AaABAg', 'snippet': {'videoId': '5Qbd0Y-KghE', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'jGyjKwyenGtoTTo4aqN0vLOiL7Y', 'id': 'Ugw51bV4A9ySSp5yULJ4AaABAg', 'snippet': {'videoId': '5Qbd0Y-KghE', 'textDisplay': '何なんだあんたは…⁉', 'textOriginal': '何なんだあんたは…⁉', 'authorDisplayName': 'マス・キリエライト', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AKedOLTKjyRcKKFQlkg3d9N17BXheHTVf2PwUNSUyKu3=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCUTZ3MGikiXSWHuVgi3rjPg', 'authorChannelId': {'value': 'UCUTZ3MGikiXSWHuVgi3rjPg'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 0, '